# Trazas para diferentes tipos de ejecuciones

### Tipos de Runs

LangSmith admite muchos tipos diferentes de Runs. Podés especificar el tipo de Run en el decorador @traceable.
Los tipos de runs son:

- LLM: Invoca un modelo de lenguaje.
- Retriever: Recupera documentos desde bases de datos u otras fuentes.
- Tool: Ejecuta acciones mediante llamadas a funciones.
- Chain: Tipo por defecto; combina múltiples runs en un proceso más grande.
- Prompt: Inyecta un prompt para ser usado con un LLM.
- Parser: Extrae datos estructurados.

### Setup

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

### Invocaciones a un LLM

LangSmith ofrece un renderizado y procesamiento especial para las trazas de LLM. Para aprovechar al máximo esta funcionalidad, tenés que registrar (loggear) tus trazas de LLM en un formato específico.

Para modelos de estilo chat, las entradas deben ser una lista de mensajes en formato compatible con OpenAI, representados como diccionarios de Python u objetos de TypeScript. Cada mensaje debe incluir las claves role y content.

La salida puede estar en cualquiera de estos formatos:
- Un diccionario/objeto que tenga la clave choices, cuyo valor sea una lista de diccionarios/objetos. Cada uno debe contener la clave message, que corresponde a un objeto mensaje con role y content.
- Un diccionario/objeto que tenga la clave message, cuyo valor sea un objeto mensaje con role y content.
- Una tupla/arreglo de dos elementos, donde el primero es el role y el segundo es el content.
- Un diccionario/objeto que contenga directamente las claves role y content.
La entrada de tu función debe llamarse messages.

También podés proveer los siguientes metadatos para ayudar a LangSmith a identificar el modelo y calcular costos. Si usás LangChain o el wrapper de OpenAI, estos campos se completan automáticamente:
- ls_provider: el proveedor del modelo, por ejemplo "openai" o "anthropic".
- ls_model_name: el nombre del modelo, por ejemplo "gpt-4o-mini" o "claude-3-opus-20240307".

In [3]:
from langsmith import traceable

inputs = [
  {"role": "system", "content": "Sos un asistente servicial."},
  {"role": "user", "content": "Me gustaría reservar una mesa para dos."},
]

output = {
  "choices": [
      {
          "message": {
              "role": "assistant",
              "content": "Claro, ¿para qué hora le gustaría reservar la mesa?"
          }
      }
  ]
}


# También se puede usar uno de los siguientes:
# output = {
#     "message": {
#         "role": "assistant",
#         "content": "Claro, ¿para qué hora le gustaría reservar la mesa?"
#     }
# }
#
# output = {
#     "role": "assistant",
#     "content": "Claro, ¿para qué hora le gustaría reservar la mesa?"
# }
#
# output = ["assistant", "Claro, ¿para qué hora le gustaría reservar la mesa?"]

@traceable(
  #  run_type="llm"
)
def chat_model(messages: list):
  return output

chat_model(inputs)

{'choices': [{'message': {'role': 'assistant',
    'content': 'Claro, ¿para qué hora le gustaría reservar la mesa?'}}]}

### Recuperación de documentos de bases de datos u otras fuentes

Muchas aplicaciones de LLM necesitan buscar documentos en bases vectoriales, grafos de conocimiento u otros tipos de índices. Las retriever traces sirven para registrar los documentos que recupera el retriever. LangSmith ofrece un renderizado especial para estos pasos de recuperación dentro de las trazas, lo que facilita entender y diagnosticar problemas de búsqueda. Para que los pasos de recuperación se muestren correctamente, hay que seguir algunos pasos simples:

1. Anotar el paso del retriever con run_type="retriever".
2. Devolver una lista de diccionarios de Python u objetos de TypeScript desde el paso del retriever.
Cada diccionario debe incluir:
    - page_content: el texto del documento.
    - type: siempre debe ser "Document".
    - metadata: un diccionario u objeto con metadatos sobre el documento, que se mostrará en la traza.

In [5]:
from langsmith import traceable

def _convert_docs(results):
  return [
      {
          "page_content": r,
          "doc_type": "Document"
          #"type": "Document",
          "metadata": {"foo": "bar"}
      }
      for r in results
  ]

@traceable(
    #run_type="retriever"
)
def retrieve_langsmith_docs(query):
  # Retriever que devuelve documentos de prueba hardcodeados.
  # En producción, esto podría ser una base de datos vectorial real u otro tipo de índice de documentos.
  contents = ["LangSmith Document contents 1", "LangSmith Document contents 2", "LangSmith Document contents 3"]
  return _convert_docs(contents)

retrieve_langsmith_docs("User query")

[{'page_content': 'LangSmith Document contents 1',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'LangSmith Document contents 2',
  'type': 'Document',
  'metadata': {'foo': 'bar'}},
 {'page_content': 'LangSmith Document contents 3',
  'type': 'Document',
  'metadata': {'foo': 'bar'}}]

### Llamada a herramientas (tools) y Chains

LangSmith tiene un renderizado especial para las Tool Calls realizadas por el modelo, para dejar claro cuándo se están utilizando las herramientas proporcionadas.

In [7]:
from langsmith import traceable
from openai import OpenAI
from typing import List, Optional
import json

openai_client = OpenAI()

@traceable(
    run_type="tool"
)
def search_web(query: str):
    return f"No se encontró ningún resultado para la consulta: {query}"

@traceable(run_type="llm")
def call_openai(
    messages: List[dict], tools: Optional[List[dict]]
) -> str:
  return openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
    tools=tools
  )

@traceable(run_type="chain")
def respond(inputs, tools):
  response = call_openai(inputs, tools)
  tool_call_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
  query = tool_call_args["query"]
  tool_response_message = {
    "role": "tool",
    "content": json.dumps({
        "query": query,
        "results": search_web(query),
    }),
    "tool_call_id": response.choices[0].message.tool_calls[0].id
  }
  inputs.append(response.choices[0].message)
  inputs.append(tool_response_message)
  output = call_openai(inputs, None)
  return output

tools = [
    {
      "type": "function",
      "function": {
        "name": "search_web",
        "description": "Buscar en la web una consulta específica",
        "parameters": {
          "type": "object",
          "properties": {
            "query": {
              "type": "string",
              "description": "La consulta que se va a buscar en la web"
            }
          },
          "required": ["query"]
        }
      }
    }
]
inputs = [
  {"role": "system", "content": "Sos un asistente servicial."},
  {"role": "user", "content": "Cómo está el clima hoy en Buenos Aires?"},
]

respond(inputs, tools)

ChatCompletion(id='chatcmpl-CdfooBhyveGCJETfbY1LFoaGJrHMP', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Lo siento, no puedo acceder a información en tiempo real sobre el clima. Te recomiendo que consultes una aplicación de clima o un sitio web confiable para obtener la información más actualizada sobre el clima en Buenos Aires.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1763571578, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=44, prompt_tokens=92, total_tokens=136, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))